# Math 182 Algorithms Final Project
### Group H: Lathem Wojno, Ashley Chang, and Cavan Stewert 

# Gale-Shapley Algorithm and its Applications/Variations
### Proposed Application: Uber Driver + Rider Stable Matching 


### Contents
1. Project Description
2. Gale-Shapely Stable Marriage Implementation

## 1. Project Description 

### Background 

The original algorithm proposed to solve the "Stable-Matching" problem was devised by David Gale and Lloyd Shapley in 1962. In their original paper titled "College Admissions and the Stability of Marriage", Gale and Shapley looked towards two of the most basic possible matching scenarios, in which they hoped to discover a systematic approach that would always find the optimal matching solution. The optimal solution, or a "stable" match, would consist of ensuring every element was a member of at least one pair, and no two unpaired elements would both prefer to be paired with one another instead of their partners. Gale and Shapley's first application of their algorithm was even simpler, because they were looking at two equal sized groups of people and only allowed every person to be apart of one single match, or in this case, marriage. Similarly, the most basic implementation of their algorithm on the college admissions process assumed that every college would match with just one prospective student in equal numbers. However, they also discussed how the algorithm could be adapted to match multiple students to each college, which would be the first of many such adaptions of their algorithm that can be used to find optimal solutions to more complex problems. 

### Gale-Shapley Stable Marriage Implementation

Before trying to adapt the Gale-Shaply algorithm to a new application, we will first implement it in its basic form following the stable-marriage problem. This will provide us with the framework needed before proceeding to our more complex problem. The first requirement to the stable marriage problem is to maintain lists of preference for both the men and the women to be matched. The algorithm proceeds by iterating through all of the women (or men) until every woman (or man) has been matched. In the case of iterating through the women, we will select the next woman on the list and iterate over her preferences until a man accepts her proposal and she becomes engaged. We will do this by placing all of the woman in a priority que. If the next most prefered man on the current woman's list is 'free', then they become engaged. If the next most prefered man is already engaged, he will choose to either reject the offer or disengage with his current match to engage with this woman. He will do this based on his own maintained preference list. Again, we will continue in this way until the woman has been matched. If a man chooses to disengage with a woman, then she is added back into the priority que, indictating that she still has to be matched and her preference list will later be reiterated over again. This implementation has a both a time and space complexity of $O(n^2)$, where n is the number of woman (or men). This is because we are assuming equal number of women and men, and maintaining a preference list for each of them of size n, corressponding to the opposite gender. The time complexity upper bound arises as there could be at most n iterations over lists of length n times some constant. 

### Proposed Application of Stable Matching Algorithm: Uber Drivers + Riders 

The main piece of our project will be attempting to simulate and find the optimal solution to matching Uber drivers with Uber app users requesting rides. In order to accomplish this, we will be first adapting the implementation of the stable marriage problem in the simplest case, where there are n drivers and n passengers and only one factor determining preference for each group. A randomized weighted graph will be used to model this scenario, and our initial focus will be to optimize this scenario for n passengers each requesting a single ride. We will then try to make our simulation more realistic by adding additional variables to our model, such as time, uber carpools, and variable number of passengers and available drivers. Our main goal will be to find an optimal solution to this problem over each time step when there are drivers available, or in other words determining who the available drivers should pick up next depending on their preferences, which will become more complex as well. However, we will also be striving to implement this with the lowest possible time and space complexity. We will also focus on programming a visualization of the algorithm running for each time step to show the locations of the drivers and passengers at each time step. 

#### Weighted Graph Model 
Our weighted graph model will act as a network of locations and roads linking these locations. Each node will be a location where either a passenger or driver can be located at when requesting a ride or waiting to be assigned to a job, respectively. Each weighted edge will be a rode with the weight indicating the distance between the two connected locations. We will maintain the weighted graph as an adjacency list, which will lower our overall time complexity in comparision to using an adjacency matrix representation. This is due to the fact that passenger and driver preference will be dependent on shortest and longest paths throughout the weighted graph model depending on their current location and the passengers indicated destination. 

#### Basic Implementation 
For our initial, basic implementation, we will first produce a randomized weighted graph and place an equal number of drivers and passengers at nodes throughout the graph. The passengers' requested destinations will also be randomized and their preference lists will be computed using an adaptation of BFS to find which drivers are nearest too them to minimize their wait time. In the basic implementation, the drivers' preference lists will the computed in a similar fashion, so to minimize the time they are driving around or waiting without getting paid. We will not be taking into account the distance of the trips requested in the basic implementation. Therefore, the stable marriage algorithm should be easy to adapt to this situation once the preference lists are built, and each passenger is linked to a driver. We will keep track of the time each driver and passenger waited to be involved in a rider in order to check if this is indeed the optimal solution. 

#### Possible Improvements 
To make our simulation more realistic and build upon our model, we will start by taking into account variables such as distance of each trip and time. Time will be implemented here by assuming each unit of weight of an edge can be traveled in one time step. This will allow us to see which drivers are busy and which drivers are available. Also, we will make the number of passengers requesting a ride variable as well. We will also run into the problem of equal preference, where a passenger is the same distance away from two drivers and would like to ride with either one, or two passengers are located at the same location and requesting a ride. Also, we will take into account the desire time a driver wants to be active for (x number of time steps) and his preference for longer distance drives in order to make more money. We could also add in another variable such as two types of drivers, one set that prefers to stay closer to 'home', or their starting location, and another set that would just prefer the longest distance drives in order to make the most money while working. Lastly, we will attempt to integrate carpools or rideshares into our simulation where a driver can pick up more than one rider at a time, and would prefer to do so if that rider is close to them, and their destination is on their current path. When taking into account ride distance, we will also have to use algorithms such as Djikstra in order to find the shortest path to a destination, another extension of BFS. Our overall goal will be to keep the time complexity of our complete algorithm in polynomial time for each time step (multiplied by a constant number of time steps for which the algorithm is run). We believe this is possible as the BFS applications involved will all be dependent on the number of edges and nodes, and even in equal preference applications of the Gale-Shapley algorithm, $O(n^4)$ time complexity can still be achieved. 

## 2. Gale-Shapely Stable Marriage Implementation

In [23]:
import heapq
import pprint
import random
import string


def create_random_preferences(n):
    """Create random preference rankings for n men and n women.
    
    Args:
        n (int): Number of men and women
        
    Returns:
        {
            'men': {'<name>': [ranked preferences]},
            'women': {'<name>': [ranked preferences]}
        },
        where each name has a list of partner preferences,
        ranked in the order of highest to lowest perference.
    """
    men = list(string.ascii_uppercase)[:n]
    women = list(string.ascii_lowercase)[:n]
    
    men_preferences = {man: random.sample(women, n) for man in men}
    women_preferences = {woman: random.sample(men, n) for woman in women}
   
    return {
        'men': men_preferences,
        'women': women_preferences
    }


def create_rank_lookup(preferences):
    """Create rank lookup for each person's preferences
        
    Args:
        preferences (dict): Men and women's ranked partner preferences
        
    Returns:
        Dictionary of all people, with each person associated with
        a dict of the form {partner: rank}.
    """
    all_preferences = preferences['men'].copy()
    all_preferences.update(preferences['women'])
    
    partner_ranks = {}
    for person in all_preferences:
        partner_ranks[person] = {
            partner: rank for rank, partner in enumerate(all_preferences[person])
        }
    return partner_ranks
        

def is_better_match(woman, man, engagements, partner_ranks):
    """Check if a woman is a better match for a man than his
        current engagement.
        
    Args:
        woman (str): Name of the first person
        man (str): Name of the second person
        engagements (dict): Each person's current engagement
        partner_ranks (dict): the dictionary of rank lookups returned
            by `create_rank_lookup()`
    
    Returns:
        True if the women is a better match for the man than his
        current engagement. False otherwise.
    """
    current_engagement = engagements[man]
    return partner_ranks[man][woman] < partner_ranks[man][current_engagement]


def create_gale_shapely_match(preferences):
    """Create stable matches (engagements) using the Gale-Shapely algorithm.
    
    Args:
        preferences (dict): Men and women's ranked partner preferences
        
    Returns:
        Engagements (dict) mapping each person to his/her engaged partner
    """
    women = preferences['women'].keys()
    men = preferences['men'].keys()
    free_women = women
    
    partner_ranks = create_rank_lookup(preferences)
    engagements = {person: None for person in women + men}
    proposals = {woman: set() for woman in women}
    
    while free_women:
        woman = free_women.pop(0)
        ordered_preferences = preferences['women'][woman]
        unproposed_men = [
            man for man in ordered_preferences
            if man not in proposals[woman]
        ]
        
        for man in unproposed_men:
            if engagements[man] == None:
                engagements = get_engaged(man, woman, engagements)
                proposals[woman].add(man)
                break
            elif is_better_match(woman, man, engagements, partner_ranks):
                mans_current_partner = engagements[man]
                engagements = unengage(man, woman, engagements)
                free_women.append(mans_current_partner)
                
                engagements = get_engaged(man, woman, engagements)
                proposals[woman].add(man)
                break
        
    return engagements
                

def get_engaged(man, woman, engagements):
    engagements[man] = woman
    engagements[woman] = man
    return engagements
    
    
def unengage(man, woman, engagements):
    engagements[man] = None
    engagements[woman] = None
    return engagements
    
    
def create_random_engagements(preferences):
    """Return random pairings for the purpose of testing
        `engagements_are_stable()`.
        
    Args:
        preferences (dict): Men and women's ranked partner preferences.
        
    Returns:
        Engagements (dict) mapping each person to their engaged partner.
    """
    engagements = {}
    women = preferences['women'].keys()
    free_women = list(random.sample(women, len(women)))
    
    for man in preferences['men']:
        woman = free_women.pop(0)
        engagements[man] = woman
        engagements[woman] = man
        
    return engagements

In [64]:
# Testing for Gale-Shapely implementation

def test_gale_shapely(n_matches, print_scenarios=True):
    """Test Gale-Shapely implementation of stable matching algorithm,
        by generating random preferences and checking stability
        of engagements.
        
    Args:
        n_partners (int): Number of man-woman matches to create
        print_scenarios (boolean): Optional. If True, preferences and
            engagements will be printed.
            
    Returns:
        None
    """
    preferences = create_random_preferences(n_matches)
    random_engagements = create_random_engagements(preferences)
    gale_shapely_engagements = create_gale_shapely_match(preferences)
    
    assert engagements_are_stable(preferences, gale_shapely_engagements)
    assert all_indidivuals_are_paired(gale_shapely_engagements)
    assert only_monogamous_engagements(gale_shapely_engagements)

    if print_scenarios:
        print_test_output(preferences, gale_shapely_engagements, random_engagements)
        
               
def engagements_are_stable(preferences, engagements, verbose=False):
    """Check whether the proposed engagements are stable.
    
    Args:
        preferences (dict): Men and women's ranked partner preferences
        engagements (dict): {'<name>': '<partner>'}, where person is mapped
            to the person he/she is engaged to
        verbose (boolean): Optional. If True, unstable matches will be printed.
            
    Returns:
        True if the engagement is stable, false otherwise.
    """
    partner_ranks = create_rank_lookup(preferences)
    women_preferences = preferences['women']
    
    for woman in women_preferences:
        engaged_partner = engagements[woman]
        
        for man in women_preferences[woman]:
            # If we find a man for whom the woman is a better match
            # before we reach her engaged partner, then we have
            # found an unstable match.
            if engaged_partner == man:
                break
            elif is_better_match(woman, man, engagements, partner_ranks):
                if verbose:
                    print(
                        "Found unstable match: {} and {}".format(woman, man) +
                        " should be engaged."
                    )
                return False
    
    return True


def all_indidivuals_are_paired(engagements):
    """Check whether all individuals have a partner.
    
    Args:
        engagements (dict): partner pairings
        
    Returns:
        True if all individuals have a partner, False otherwise.
    """
    return all([partner is not None for person, partner in engagements.items()])


def only_monogamous_engagements(engagements):
    """Check that no individual is engagement more than once.
    
    Args:
        engagements (dict): partner pairings
        
    Returns:
        True if all individuals are engaged at most once, False otherwise.
    """
    n_women_engaged = len(set(engagements.keys()))
    n_men_engaged = len(set(engagements.values()))
    return n_women_engaged == n_men_engaged 


def print_test_output(preferences, gale_shapely_engagements, random_engagements):
    """Print testing scenarios.
    
    Args:
        preferences (dict): Ordered preferences of each partner
        gale_shapely_engagements (dict): partner pairings
        random_engagements (dict): partner pairings
        
    Returns:
        None
    """
    print('Preferences')
    print('-' * 50)
    pprint.pprint(preferences)

    print('\nGale-Shapely Engagements')
    print('-' * 50)
    stability = engagements_are_stable(preferences, gale_shapely_engagements)
    print("Is stable engagement: {}".format(stability))
    pprint.pprint(gale_shapely_engagements)

    print('\nRandom Engagements')
    print('-' * 50)
    stability = engagements_are_stable(preferences, random_engagements)
    print("Is stable engagement: {}".format(stability))
    pprint.pprint(random_engagements)

In [67]:
test_gale_shapely(10)

Preferences
--------------------------------------------------
{'men': {'A': ['j', 'd', 'b', 'g', 'e', 'h', 'a', 'f', 'i', 'c'],
         'B': ['h', 'g', 'd', 'e', 'c', 'f', 'a', 'b', 'i', 'j'],
         'C': ['d', 'h', 'f', 'c', 'i', 'b', 'g', 'j', 'a', 'e'],
         'D': ['b', 'a', 'i', 'd', 'g', 'e', 'f', 'j', 'c', 'h'],
         'E': ['g', 'd', 'h', 'b', 'f', 'e', 'c', 'a', 'j', 'i'],
         'F': ['i', 'f', 'e', 'h', 'a', 'j', 'd', 'g', 'c', 'b'],
         'G': ['c', 'i', 'b', 'h', 'j', 'f', 'e', 'g', 'a', 'd'],
         'H': ['b', 'a', 'c', 'g', 'h', 'i', 'e', 'f', 'j', 'd'],
         'I': ['b', 'g', 'j', 'd', 'a', 'f', 'c', 'i', 'h', 'e'],
         'J': ['b', 'a', 'c', 'h', 'd', 'i', 'g', 'f', 'j', 'e']},
 'women': {'a': ['C', 'B', 'I', 'A', 'H', 'D', 'F', 'J', 'E', 'G'],
           'b': ['B', 'F', 'H', 'G', 'C', 'I', 'J', 'A', 'E', 'D'],
           'c': ['A', 'B', 'D', 'F', 'H', 'I', 'J', 'E', 'G', 'C'],
           'd': ['B', 'E', 'D', 'F', 'J', 'C', 'A', 'H', 'G', 'I'],
    